In [2]:
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.model_selection import StratifiedKFold

In [5]:
tte_data = pd.read_csv('dataset/data_censored.csv')
cols_to_drop = ['age_s', 'id', 'censored', 'eligible']
tte_data = tte_data.drop(columns=cols_to_drop)
print(tte_data.head(10))

   period  treatment  x1        x2  x3        x4  age  outcome
0       0          1   1  1.146148   0  0.734203   36        0
1       1          1   1  0.002200   0  0.734203   37        0
2       2          1   0 -0.481762   0  0.734203   38        0
3       3          1   0  0.007872   0  0.734203   39        0
4       4          1   1  0.216054   0  0.734203   40        0
5       5          1   0 -0.057482   0  0.734203   41        0
6       0          0   1 -0.802142   0 -0.990794   26        0
7       1          1   1 -0.983030   0 -0.990794   27        0
8       2          1   0  0.399388   0 -0.990794   28        0
9       3          0   0  1.835085   0 -0.990794   29        0


In [4]:
treatment = 'treatment'
outcome = 'outcome'